In [1]:
import gym
env_name = 'MountainCar-v0'
env = gym.make(env_name)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [2]:
from tensorflow.python.ops import variables
from tensorflow.python.framework import ops
from scipy.stats import rankdata

def rescale_to_normal(array):
    return (array - np.mean(array))/ np.std(array)
        
class EL_optimize(object):
    def __init__(self, sigma = 0.1, alpha = 1e-3,beta = 1e-2,mini_batch_size = 30):
        self.alpha = tf.constant(alpha)
        self.beta = tf.constant(1-beta)
        self.sigma = tf.constant(sigma)
        self.mini_batch_size = mini_batch_size
        self.current_ind = tf.placeholder("int32")
        self.update_weights = tf.placeholder("float32",shape = [mini_batch_size])
        self.fitness_shape_rank_vals = [np.max([0,np.log(self.mini_batch_size/2.0+1)-np.log(ind)]) for ind in np.arange(1,(self.mini_batch_size+1))]
        self.fitness_shape_rank_vals = self.fitness_shape_rank_vals/np.sum(self.fitness_shape_rank_vals)
        self.set_links_to_vars()

    def set_links_to_vars(self):
        self.noise_vars = []
        self.new_noise_vals_ops = []
        self.set_noise_ops = []
        self.copy_state_ops = []
        self.reset_state_ops = []
        self.update_based_on_reward_ops = []
        
        self.vars = variables.trainable_variables() + ops.get_collection(ops.GraphKeys.TRAINABLE_RESOURCE_VARIABLES)
        for var in self.vars:
            noise_var = tf.random_normal([self.mini_batch_size] + var.get_shape().as_list())
            current_noise_vals = tf.Variable(tf.zeros(noise_var.get_shape()))
            self.new_noise_vals_ops.append(current_noise_vals.assign(noise_var))
            self.noise_vars.append(current_noise_vals)
            copy_var = tf.Variable(var.initialized_value())
            
            self.set_noise_ops.append(var.assign(copy_var + self.sigma * current_noise_vals[self.current_ind]))
            self.copy_state_ops.append(copy_var.assign(var))
            self.reset_state_ops.append(var.assign(copy_var))
            
            self.update_based_on_reward_ops.append(var.assign(copy_var*self.beta + tf.tensordot(self.update_weights, current_noise_vals,axes=[0,0])))
            
    def calc_update_based_on_reward(self,rewards):
        
        self.calc_weights = self.alpha.eval()/(self.mini_batch_size * self.sigma.eval()) * rescale_to_normal(rewards)
    
    def calc_update_based_on_highest_reward(self,rewards):
        
        self.calc_weights = self.sigma.eval()*np.array([(1.0 if val == np.max(rewards) else 0) for val in np.array(rewards)])
    
    def calc_update_fitness_shaping(self,rewards):
        
        k = self.mini_batch_size - rankdata(rewards_t, method='ordinal')
        self.calc_weights = (self.alpha.eval()/self.sigma.eval())*self.fitness_shape_rank_vals[k] 

//anaconda/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
import tensorflow as tf
import tflearn
import numpy as np
tf.reset_default_graph() # THIS IS NECESSARY BEFORE MAKING NEW SESSION TO STOP IT ERRORING!!
try:
    sess
except:
    pass
else:
    sess.close()
    del sess
sess = tf.InteractiveSession()

n_inputs = env.observation_space.shape[0]
n_hidden = 8  # it's a simple task, we don't need more than this
n_hlayers = 3
n_outputs = env.action_space.n # only outputs the probability of accelerating left
initializer = tf.contrib.layers.variance_scaling_initializer()

# 2. Build the neural network
X = tf.placeholder(tf.float32, shape=[None, n_inputs])
Y = tf.placeholder(tf.int32, shape=[None])

layer = X
for _ in range(n_hlayers):
    layer = tf.layers.dense(layer, n_hidden, activation=tf.nn.relu,
                         kernel_initializer=initializer)

logits = tf.layers.dense(layer, n_outputs,
                          kernel_initializer=initializer)
action = tf.argmax(logits, 1)
    
saver = tf.train.Saver()

el = EL_optimize(alpha = 0.01,beta=1e-3,sigma = 0.1,mini_batch_size=30)
el.set_links_to_vars()
tf.global_variables_initializer().run()



In [4]:
observation = env.reset()
logs, act = sess.run([logits,action], feed_dict={X: np.expand_dims(observation,axis=0)})
print(logs)
print(act[0])
observation, reward, done, info = env.step(act[0])
print(observation)
print(reward)

[[ 0.08720301 -0.70554346  0.9897891 ]]
2
[-0.46818159  0.00059065]
-1.0


In [5]:
env = gym.make(env_name)

render = False

i = 0
max_t = 400

# sess.run(el.alpha.assign(0.01))

while i < 1000:
    sess.run(el.copy_state_ops)
    sess.run(el.new_noise_vals_ops)

    rewards_t = np.zeros([el.mini_batch_size])
    obs_std = np.zeros([el.mini_batch_size,env.observation_space.shape[0]])
    
    for j in range(el.mini_batch_size):
        observation = env.reset()
        obs_t = np.array([observation])
        sess.run(el.set_noise_ops, feed_dict={el.current_ind : j})
        reward_t = 0
        for t in range(max_t):
            if render:
                env.render()
            obs_t = np.append(obs_t,[observation],axis = 0)
            
            act = sess.run(action, feed_dict={X: np.expand_dims(observation,axis=0)})[0]
            observation, reward, done, info = env.step(act)
            reward_t += reward
            if done or (t == max_t-1):
#                 obs_std[j,:] = np.std(obs_t,axis=0)
                rewards_t[j] = reward_t
                break
    print('mini batch results')
        
#     if np.std(rewards_t) == 0:
#         print('No reward variance, adding obs std as reward')
#         rewards_t = np.mean(np.array([(obs) for obs in obs_std.T]),axis=0)

    print(np.mean(rewards_t))
#     print(rewards_t)
    
#     el.calc_update_based_on_reward(rewards_t)
    el.calc_update_fitness_shaping(rewards_t)
#     print(el.calc_weights)
    sess.run(el.update_based_on_reward_ops, feed_dict={el.update_weights : el.calc_weights})
        
    i += 1
                
            
# saver.save(sess, "./my_policy_net_basic.ckpt")

env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-198.43333333333334
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini

mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch r

mini batch results
-200.0
mini batch results
-190.56666666666666
mini batch results
-199.83333333333334
mini batch results
-197.2
mini batch results
-194.23333333333332
mini batch results
-196.4
mini batch results
-200.0
mini batch results
-198.3
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-200.0
mini batch results
-198.83333333333334
mini batch results
-200.0
mini batch results
-199.13333333333333
mini batch results
-199.23333333333332
mini batch results
-195.66666666666666
mini batch results
-197.1
mini batch results
-191.8
mini batch results
-193.43333333333334
mini batch results
-195.36666666666667
mini batch results
-194.46666666666667
mini batch results
-193.56666666666666
mini batch results
-199.33333333333334
mini batch results
-195.2
mini batch results
-195.93333333333334
mini batch results
-195.8
mini batch results
-196.23333333333332
mini batch results
-194.96666666666667
mini batch results
-197.43333333333334
mini batch results
-200.0
mini batch r

mini batch results
-176.1
mini batch results
-182.7
mini batch results
-183.6
mini batch results
-178.06666666666666
mini batch results
-185.16666666666666
mini batch results
-188.73333333333332
mini batch results
-198.1
mini batch results
-188.73333333333332
mini batch results
-188.5
mini batch results
-182.66666666666666
mini batch results
-192.53333333333333
mini batch results
-190.43333333333334
mini batch results
-190.5
mini batch results
-190.5
mini batch results
-190.23333333333332
mini batch results
-178.9
mini batch results
-180.46666666666667
mini batch results
-189.3
mini batch results
-182.6
mini batch results
-192.83333333333334
mini batch results
-191.43333333333334
mini batch results
-188.16666666666666
mini batch results
-182.36666666666667
mini batch results
-189.33333333333334
mini batch results
-190.66666666666666
mini batch results
-183.2
mini batch results
-185.73333333333332
mini batch results
-180.4
mini batch results
-184.86666666666667
mini batch results
-189.6

In [91]:
# saver.save(sess, "./my_policy_net_basic.ckpt")

'./my_policy_net_basic.ckpt'

In [11]:
rewardsum = 0
env = gym.make(env_name)
obs = env.reset()
with tf.Session() as sess:
    saver.restore(sess, "./my_policy_net_basic.ckpt")
    for step in range(1000):
        env.render()
        action_val = action.eval(feed_dict={X: obs.reshape(1, n_inputs)})
        obs, reward, done, info = env.step(action_val[0])
        rewardsum += reward
        if done:
            break
env.close()
print(rewardsum)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO:tensorflow:Restoring parameters from ./my_policy_net_basic.ckpt
174.1687398409288
